# Onchain Credit Risk Analysis - Overview 

### 1) What is the problem we are trying to solve? 

Assess the likelihood that a crypto address will repay borrowed funds. 

Two important factors: ability and willingness.

This analysis attempts to improve understanding around on chain borrower's willingness to repay borrowed funds. Typical reason for repaying borrowed funds in real life revolve around social reputation and a verified individual's ability to access borrowed funds in the future. 

### 2) What is considered a bad outcome? 

Without overcollateralizion, the psudonymous nature of blockchains allows any sybil actor to spin up an address, generate positive activity, access credit, and abscond with the borrowed funds. 

In the current on chain borrowing paradigm, overcollateralization is required and loan defaults are swiftly resolved through open auctioning of the collateral (i.e., liquidations). 

For the purposes of this analysis, a bad outcome is definded as any lending agreement that results in a liquidation (if liquidation, result = 0, else result = 1). Considered a credit default event. 

### 3) Can we avoid bad outcomes?

Stopping short of full off chain identity verification, is there a way to assess a wallet address's (i.e., borrower's) willingness to repay borrowed funds? 

__Thesis__

The social history of a wallet can tell us how much of the user's true identity is invested in a wallet address. Wallet addresses with greater identity investment are more willing to adhere to loan terms--most importantly in order to preserve the social reputation of the wallet address, as well to preserve the ability to access similar terms in the future.

__Thesis Weaknesses__

There is not currently a system for penalizing wallets that have defaulted, and thus there is no social or credit profile degradation threat to disincentivize blockchain actors from misbehaving.  

### 4) Methodology

Use Aave v1 lender and borrower data to build a simplified understanding of an address's financial profile. 
> Assumption: Aave v1 activity portrays the address's complete financial profile.

Use the address's social activity data to assess the wallet owner's personal investment into this address's identity. 
> Assumption: Wallet addresses with greater identity investment are more willing to adhere to loan terms. Importantly, adhering to loan terms means avoiding liquidation.

A single liquidation is considered a strict loan default event.
> Assumption: liquidations are considered a terminal event and strict defualt has occured. 

### 5) Dataset

Most recent 250 active address on Aave v1, prior to freezing the protocol in November 2022. 

For those 250 addresses, the current oustanding loan balance and the most recent 1,000 transactions across the following protocol activities: (1) Deposits, (2) Borrows, (3) Flashloans, (4) Withdrawals, (5) Loan Repayments, (6) Liquidations 

### 6) Features & Target

__Features__:

 - Total deposits in USD
 - Total borrowings outstanding in USD
 - Ratio of total liquidated collateral to total borrowings (USD)
 - Total flashloans (USD)
 - Prior repayments made (yes/no)
 - Prior liquidation event
 - Address currently resolved to ENS .eth name
 - Address currently resolved to ENS .eth name with sub domains
 - Address resolved ENS name age in days
 - Address currently owns a Lens profile on Polygon
 - Lens profile name contains 'DAO'
 - Lens follower count
 - Lens following count
 - Lens publication count
 - Lens collection count
 - Address is signer on a gnosis safe (polygon only--safe's ethereum subgraph returned indexing errors)
 - Address holds a bufficorn or $SPORK tokens
 - Address has donated to gitcoin grants
 - Address has an ethCC admission ticket 

__Target__: 

- result

### 7) Analysis & Estimation of Feature Usefulness
The social feature set did not provide much signal. Perhaps we need more data. More likely, the industry has yet to systematically describe wallets using on chain social activities, which as previously predicted leave little incentive for wallet addresses to behave. 

Incomplete picture of user assets and liabilities given the queries do not crawl each user's entire history with aave.

Problems with the below, since liquidations have already occured. Would need future activity to see if past liquidations predict future liquidations.
 - Ratio of total liquidated collateral to total borrowings (USD)

__To Do:__

Get data for the following features

 - Address holds a bufficorn or $SPORK tokens
 - Address has donated to gitcoin grants
 - Address has an ethCC admission ticket 


### 8) Modeling

Simple decision tree to check accuracy and feature importance

### 9) Results

The most important feature in my final analysis set was flash loan amount (USD). 

The feature was negatively correlated with the target variable. Addresses with higher values of flash loans were more likely to have a liquidation event. 



In [596]:
# libraries
import asyncio
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import json
import numpy as np
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
import datetime
import time
import requests as r

# Set the display format for floating point numbers
# Set the maximum width of the columns in the output of DataFrame.info()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_colwidth', 100)

In [281]:
# set main params for all aave queries
graph_url = "https://api.thegraph.com/subgraphs/name/aave/protocol-multy-raw"
transport = AIOHTTPTransport(url=graph_url)

In [607]:
# last 1000 transactions prior to Oct. 31, 2022
# borrow asynchronous subgraph call - get data
async def txLatestMain(startRow, endRow, transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query latestUserTransactions {{
                    userTransactions(where: {{timestamp_lte: 1667246161}}, skip:{startRow}, first:{endRow},, orderBy: timestamp, orderDirection: desc){{
                        id
                        timestamp
                        user {{
                            id
                        }}
                    }}
                }}
        '''.format(startRow = startRow, endRow = endRow, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def appendList(startRow, endRow, transport):
    result_json = await txLatestMain(startRow, endRow, transport)
    return result_json["userTransactions"]

In [608]:
# latest 250 wallet addresses
# empty list to capture main graphql calls
return_list = []

# set skip rows & first rows params
incrementRow = 1000
startRow = 0
endRow = incrementRow

# get k rows by looping n times
for k in range(2):
    this_list = await appendList(startRow, endRow, transport)
    # iterate the params
    startRow = startRow + incrementRow
    return_list.append(this_list)

# flaten list
return_list = [item for sublist in return_list for item in sublist]
print(f'records returned from latest tx subgraph is {len(return_list)} records')

# convert list of json to df
df_latest_txs = pd.json_normalize(return_list)

# convert data types, reorder columns, and convert wei values
# filter out aave contracts
df_latest_txs = df_latest_txs[~df_latest_txs['user.id'].isin(['0xbcc4371cc40592794bf5b727c17cf7de37ac180a','0xd6ad7a6750a7593e092a9b218d66c0a814a3436e'])]
df_latest_txs['timestamp'] = pd.to_datetime(df_latest_txs['timestamp'], unit='s')
df_latest_txs.sort_values(by='timestamp', ascending=False, inplace=True)

# last 100 txs
df_last_hundred = df_latest_txs[:100]

# get unique addresses out of last 250 user list
latest_users = df_latest_txs['user.id'].unique()[:250]

# convert to string
latest_user_query_string = ','.join([f'"{str(elem)}"' for elem in latest_users])

# how many records?
print(f"now getting aave history for {len(latest_users)} users")

records returned from latest tx subgraph is 2000 records
now getting aave history for 250 users


In [610]:
# create last 250 user df
df_users = pd.DataFrame(latest_users, columns=['user.id'])
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user.id  250 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


In [611]:
# aToken query
# borrow asynchronous subgraph call - get data
async def aTokenMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query userReserves {{
                userReserves(where: {{user: "{userList}"}}, first: 1) {{
                    user {{
                        id
                    }}
                    aTokenBalanceHistory(first: 1, orderBy: timestamp, orderDirection: desc) {{
                        balance
                        userReserve {{
                            reserve {{
                                decimals
                                symbol
                                price {{
                                    priceInEth
                                }}
                            }}
                        }}
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main
async def aTokenCallMain(userList,transport):
    result_json = await aTokenMain(userList,transport)
    return result_json["userReserves"]

In [613]:
aToken_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await aTokenCallMain(user, transport)
    # iterate the params
    aToken_list.append(this_list)

# flaten list
aToken_list = [item for sublist in aToken_list for item in sublist]
print(f'combined list is {len(aToken_list)} records')

combined list is 250 records


In [614]:
# deposits
# borrow asynchronous subgraph call - get data
async def userDepositMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query userReserves {{
                deposits(where: {{user: "{userList}"}}, first: 1000, orderBy: timestamp, orderDirection: desc) {{
                    id
                    amount
                    timestamp
                    user {{
                        id
                    }}
                    reserve {{
                        symbol
                        decimals
                            price {{
                                priceInEth
                            }}
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def depositCallMain(userList,transport):
    result_json = await userDepositMain(userList,transport)
    return result_json["deposits"]

In [615]:
# deposit list
# empty list to capture main graphql calls
deposit_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await depositCallMain(user, transport)
    # iterate the params
    deposit_list.append(this_list)

# flaten list
deposit_list = [item for sublist in deposit_list for item in sublist]
print(f'combined list is {len(deposit_list)} records')

combined list is 2676 records


In [616]:
# borrows
# borrow asynchronous subgraph call - get data
async def userBorrowMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query userReserves {{
                borrows(where: {{user: "{userList}"}}, first: 1000, orderBy: timestamp, orderDirection: desc) {{
                    id
                    timestamp
                    amount
                    user {{
                        id
                    }}
                    reserve {{
                        symbol
                        decimals
                        price {{
                            priceInEth
                        }}
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def borrowCallMain(userList,transport):
    result_json = await userBorrowMain(userList,transport)
    return result_json["borrows"]

In [617]:
# borrow list
# empty list to capture main graphql calls
borrow_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await borrowCallMain(user, transport)
    # iterate the params
    borrow_list.append(this_list)

# flaten list
borrow_list = [item for sublist in borrow_list for item in sublist]
print(f'combined list is {len(borrow_list)} records')

combined list is 1531 records


In [618]:
# flashloans
# main asynchronous subgraph call - get data
async def flashMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query_script = '''
            query flashLoans {{
                    flashLoans(where: {{user: "{userList}"}}, first:1000, orderBy: timestamp, orderDirection: desc){{
                    id
                    timestamp
                    amount
                    reserve {{
                        decimals
                    }}
                }}
                }}
        '''.format(userList = userList, transport = transport)
        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def callFlashMain(userList, transport):
    result_json = await flashMain(userList, transport)
    return result_json["flashLoans"]

In [619]:
# Flash list
# empty list to capture main graphql calls
flash_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await borrowCallMain(user, transport)
    # iterate the params
    flash_list.append(this_list)

# flaten list
flash_list = [item for sublist in flash_list for item in sublist]
print(f'combined list is {len(flash_list)} records')

combined list is 1531 records


In [620]:
# repays
# borrow asynchronous subgraph call - get data
async def userRepayMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query repays {{
                repays(where: {{user: "{userList}"}}, first: 1000, orderBy: timestamp, orderDirection: desc) {{
                    id
                    user {{
                        id
                    }}
                    timestamp
                    amountAfterFee
                    reserve {{
                        symbol
                        decimals
                        price {{
                            priceInEth
                        }}
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def repayCallMain(userList,transport):
    result_json = await userRepayMain(userList,transport)
    return result_json["repays"]

In [621]:
# repay list
# empty list to capture main graphql calls
repay_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await repayCallMain(user, transport)
    # iterate the params
    repay_list.append(this_list)

# flaten list
repay_list = [item for sublist in repay_list for item in sublist]
print(f'combined list is {len(repay_list)} records')

combined list is 869 records


In [622]:
# withdrawals
# borrow asynchronous subgraph call - get data
async def userWithdrawalMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query redeemUnderlyings {{
                    redeemUnderlyings(where: {{user: "{userList}"}}, first:1000, orderBy: timestamp, orderDirection: desc) {{
                        id
                        amount
                        timestamp
                        user {{
                            id
                        }}
                        reserve {{
                            symbol
                            decimals
                            price {{
                                priceInEth
                            }}
                        }}
                    }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def withdrawalCallMain(userList,transport):
    result_json = await userWithdrawalMain(userList,transport)
    return result_json["redeemUnderlyings"]

In [623]:
# withdrawal list
# empty list to capture main graphql calls
withdrawal_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await withdrawalCallMain(user, transport)
    # iterate the params
    withdrawal_list.append(this_list)

# flaten list
withdrawal_list = [item for sublist in withdrawal_list for item in sublist]
print(f'combined list is {len(withdrawal_list)} records')

combined list is 3085 records


In [624]:
# liquidations
# borrow asynchronous subgraph call - get data
async def userLiquidationMain(userList,transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query liquidationCalls {{
                liquidationCalls(where: {{user: "{userList}"}}, first: 1000, orderBy: timestamp, orderDirection: desc) {{
                    id
                    user {{
                        id
                    }}
                    timestamp
                    collateralAmount
                    collateralReserve {{
                        symbol
                        decimals
                        price {{
                            priceInEth
                        }}
                    }}
                    principalAmount
                    principalReserve {{
                        symbol
                        decimals
                        price {{
                            priceInEth
                        }}
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get 1k txs in each call
async def liquidationCallMain(userList,transport):
    result_json = await userLiquidationMain(userList,transport)
    return result_json["liquidationCalls"]

In [625]:
# liquidation list
# empty list to capture main graphql calls
liquidation_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await liquidationCallMain(user, transport)
    # iterate the params
    liquidation_list.append(this_list)

# flaten list
liquidation_list = [item for sublist in liquidation_list for item in sublist]
print(f'combined list is {len(liquidation_list)} records')

combined list is 219 records


In [626]:
# convert all list of json to df
# note, deposit and borrow list queries pulled directly from the activity subgraphs and not the user reserve subgraph. Hence, the query results are different.
df_user_aTokens = pd.json_normalize(aToken_list,record_path=['aTokenBalanceHistory'],meta=[['user', 'id']],record_prefix='aTokens.',errors='ignore')
df_user_deposits = pd.json_normalize(deposit_list)
df_user_borrows = pd.json_normalize(borrow_list)
df_user_flash = pd.json_normalize(flash_list)
df_user_withdrawals = pd.json_normalize(withdrawal_list)
df_user_repays = pd.json_normalize(repay_list)
df_user_liquidations = pd.json_normalize(liquidation_list)

In [627]:
df_user_deposits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        2676 non-null   object
 1   amount                    2676 non-null   object
 2   timestamp                 2676 non-null   int64 
 3   user.id                   2676 non-null   object
 4   reserve.symbol            2676 non-null   object
 5   reserve.decimals          2676 non-null   int64 
 6   reserve.price.priceInEth  2676 non-null   object
dtypes: int64(2), object(5)
memory usage: 146.5+ KB


In [657]:
# summarize last 100 txs

# merge all dfs back to the latest tx df
# first create tx label column for each df
df_user_deposits['tx_type'] = 'deposit'
df_user_borrows['tx_type'] = 'borrow'
df_user_withdrawals['tx_type'] = 'withdrawal'
df_user_liquidations['tx_type'] = 'liquidation'
df_user_flash['tx_type'] = 'flash_loan'
df_user_repays['tx_type'] = 'repay'

df_last_hundred_summary = pd.merge(df_last_hundred, df_user_deposits[['id','tx_type']], how="left", left_on=["id"], right_on=["id"]).merge(df_user_borrows[['id','tx_type']], how="left", left_on=['id'], right_on=['id']) \
                            .merge(df_user_withdrawals[['id','tx_type']], how="left", left_on=['id'], right_on=['id']) \
                            .merge(df_user_liquidations[['id','tx_type']], how="left", left_on=['id'], right_on=['id']) \
                            .merge(df_user_flash[['id','tx_type']], how="left", left_on=['id'], right_on=['id']) \
                            .merge(df_user_repays[['id','tx_type']], how="left", left_on=['id'], right_on=['id'])

In [658]:
# borrow/flash loan
df_last_hundred_summary.columns = ['id','timestamp','user.id','deposit','borrow','withdrawal','liquidation','flash','repay']
df_last_hundred_summary['borrow_flash_loan'] = np.where((~df_last_hundred_summary['borrow'].isna())&(~df_last_hundred_summary['flash'].isna()), 'borrow/flash', pd.NA)
df_last_hundred_summary = df_last_hundred_summary[['id','timestamp','user.id','deposit','borrow_flash_loan','borrow','withdrawal','liquidation','flash','repay']]

In [659]:
# find tx type from new columns and then drop uneccessary columns
df_last_hundred_summary['tx_type'] = df_last_hundred_summary.iloc[:, 3:].bfill(axis=1).iloc[:, 0].fillna('unknown')
df_last_hundred_summary.drop(['deposit','borrow_flash_loan','borrow','withdrawal','liquidation','flash','repay'], axis=1, inplace=True)

In [664]:
df_last_hundred_summary.groupby(['tx_type'])['tx_type'].count().sort_values(ascending=False)

tx_type
withdrawal      61
deposit         20
borrow/flash    11
repay            6
liquidation      1
unknown          1
Name: tx_type, dtype: int64

In [ ]:
# get unique token list for making call to coingecko api
token_list = np.unique(np.concatenate((df_user_deposits['reserve.symbol'].unique(), \
    df_user_borrows['reserve.symbol'].unique(), \
        df_user_withdrawals['reserve.symbol'].unique(), \
        df_user_repays['repay.reserve.symbol'].unique(), \
            df_user_liquidations['liquidation.collateralReserve.symbol'].unique(), \
                df_user_liquidations['liquidation.principalReserve.symbol'].unique()), axis=0))

In [202]:
# get coingecko list of token ids
cg_token_list_response = r.get('https://api.coingecko.com/api/v3/coins/list?include_platform=false')
if cg_token_list_response:
    cg_token_list = cg_token_list_response.text
else:
    print('coingecko api error')
# convert json string to df
df_cg_token_list = pd.json_normalize(json.loads(cg_token_list))
df_cg_token_list['symbol'] = df_cg_token_list['symbol'].str.upper()
df_cg_token_list = df_cg_token_list[(df_cg_token_list['symbol'].isin(token_list))&(df_cg_token_list['name']!='UNICORN')&(df_cg_token_list['id']!='force-bridge-usdc')]

In [81]:
# must set missing tokens to $1
np.setdiff1d(token_list, df_cg_token_list)

array(['UniDAI', 'UniUSDT'], dtype=object)

In [204]:
# get last 4 years token prices for each token in dataset
# get historical ETH/USD so we can convert user activity 
unix_today = time.mktime(datetime.datetime.now().timetuple())
token_usd_list = []
for index, row in df_cg_token_list.iterrows():
    # jan 1 2019 through current date
    token_usd_endpoint = f"https://api.coingecko.com/api/v3/coins/{row['id']}/market_chart/range?vs_currency=usd&from=1546315505&to={unix_today}"
    token_usd_prices_response = r.get(token_usd_endpoint)
    if token_usd_prices_response:
        token_usd_prices = token_usd_prices_response.text
    else:
        print(token_usd_prices_response.status_code)
        print(token_usd_endpoint)
    time.sleep(7)
    token_usd_list.append({row['symbol']: json.loads(token_usd_prices)['prices']})

In [ ]:
# temp list to use for normalizing the data
flat_list = []
for item in token_usd_list:
    for key, value in item.items():
        # add token symbol to each list
        temp_list_a = value
        temp_list_a = [sublist + [key] for sublist in temp_list_a]
        flat_list.append(temp_list_a)
# flatten the list
flat_list = [item for sublist in flat_list for item in sublist]
# convert list to df
df_token_usd = pd.DataFrame(flat_list, columns=['timestamp','token_usd','token'])
df_token_usd['timestamp'] = pd.to_datetime(df_token_usd['timestamp'], unit='ms', origin='unix').dt.date
latest_token_usd = df_token_usd.sort_values(by=['timestamp'], ascending=False).drop_duplicates(subset='token')
df_token_usd.info()

In [665]:
# summarize aTokens
# first get usd prices
df_user_aTokens['aTokensAmountToken'] = (df_user_aTokens['aTokens.balance'].astype('float') / (10**df_user_aTokens['aTokens.userReserve.reserve.decimals']))
df_user_aTokens = pd.merge(df_user_aTokens, latest_token_usd[['token','token_usd']], how='left', left_on='aTokens.userReserve.reserve.symbol', right_on='token').drop('token',axis=1)
# df_user_aTokens['aTokenAmountETH'] = (df_user_aTokens['aTokensAmountToken'] * (df_user_aTokens['aTokens.userReserve.reserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_aTokens['aTokenAmountUSD'] = df_user_aTokens['aTokensAmountToken'] * df_user_aTokens['token_usd']
# summarize
df_user_aToken_sum = df_user_aTokens[['user.id','aTokenAmountUSD']].groupby(['user.id']).sum().reset_index()

In [666]:
# summarize user deposits 
# first get usd prices
df_user_deposits['depositAmountToken'] = (df_user_deposits['amount'].astype('float') / (10**df_user_deposits['reserve.decimals']))
# df_user_deposits['depositAmountETH'] = (df_user_deposits['depositAmountToken'] * (df_user_deposits['reserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_deposits['timestamp'] = pd.to_datetime(df_user_deposits['timestamp'], unit='s', origin='unix').dt.date
df_user_deposits = pd.merge(df_user_deposits, df_token_usd, how='left', left_on=['timestamp','reserve.symbol'], right_on=['timestamp','token'])
df_user_deposits['depositAmountUSD'] = df_user_deposits['depositAmountToken'] * df_user_deposits['token_usd']
# summarize
df_user_deposit_sum = df_user_deposits[['user.id','depositAmountUSD']].groupby(['user.id']).sum().reset_index()

In [667]:
# summarize user borrows
df_user_borrows['borrowAmountTokens'] = (df_user_borrows['amount'].astype('float') / (10**df_user_borrows['reserve.decimals'])) 
# df_user_borrows['borrowAmountETH'] = (df_user_borrows['borrowAmountTokens'] * (df_user_borrows['reserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_borrows['timestamp'] = pd.to_datetime(df_user_borrows['timestamp'], unit='s', origin='unix').dt.date
df_user_borrows = pd.merge(df_user_borrows, df_token_usd, how='left', left_on=['timestamp','reserve.symbol'], right_on=['timestamp','token'])
df_user_borrows['borrowAmountUSD'] = df_user_borrows['borrowAmountTokens'] * df_user_borrows['token_usd']
# summarize
df_user_borrow_sum = df_user_borrows[['user.id','borrowAmountUSD']].groupby(['user.id']).sum().reset_index()

In [668]:
# summarize user borrows
df_user_flash['flashAmountTokens'] = (df_user_flash['amount'].astype('float') / (10**df_user_flash['reserve.decimals'])) 
# df_user_borrows['borrowAmountETH'] = (df_user_borrows['borrowAmountTokens'] * (df_user_borrows['reserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_flash['timestamp'] = pd.to_datetime(df_user_flash['timestamp'], unit='s', origin='unix').dt.date
df_user_flash = pd.merge(df_user_flash, df_token_usd, how='left', left_on=['timestamp','reserve.symbol'], right_on=['timestamp','token'])
df_user_flash['flashAmountUSD'] = df_user_flash['flashAmountTokens'] * df_user_flash['token_usd']
# summarize
df_user_flash_sum = df_user_flash[['user.id','flashAmountUSD']].groupby(['user.id']).sum().reset_index()

In [669]:
# summarize withdrawals 
# first get usd prices
df_user_withdrawals['withdrawalAmountToken'] = (df_user_withdrawals['amount'].astype('float') / (10**df_user_withdrawals['reserve.decimals']))
# df_user_withdrawals['withdrawalAmountETH'] = (df_user_withdrawals['withdrawalAmountToken'] * (df_user_withdrawals['reserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_withdrawals['timestamp'] = pd.to_datetime(df_user_withdrawals['timestamp'], unit='s', origin='unix').dt.date
df_user_withdrawals = pd.merge(df_user_withdrawals, df_token_usd, how='left', left_on=['timestamp','reserve.symbol'], right_on=['timestamp','token'])
df_user_withdrawals['withdrawalAmountUSD'] = df_user_withdrawals['withdrawalAmountToken'] * df_user_withdrawals['token_usd']
# summarize
df_user_withdrawal_sum = df_user_withdrawals[['user.id','withdrawalAmountUSD']].groupby(['user.id']).sum().reset_index()

In [670]:
# summarize repays
# first get usd prices
df_user_repays['repayAmountTokens'] = (df_user_repays['amountAfterFee'].astype('float') / (10**df_user_repays['reserve.decimals']))
# df_user_repays['repayAmountETH'] = (df_user_repays['repayAmountTokens'] * (df_user_repays['repay.reserve.price.priceInEth'].astype('float') /  (10**18))) 
df_user_repays['timestamp'] = pd.to_datetime(df_user_repays['timestamp'], unit='s', origin='unix').dt.date
df_user_repays = pd.merge(df_user_repays, df_token_usd, how='left', left_on=['timestamp','reserve.symbol'], right_on=['timestamp','token'])
df_user_repays['repayAmountUSD'] = df_user_repays['repayAmountTokens'] * df_user_repays['token_usd']
# summarize
df_user_repay_sum = df_user_repays[['user.id','repayAmountUSD']].groupby(['user.id']).sum().reset_index()

In [671]:
# summarize liquidations
# first get usd prices
df_user_liquidations['timestamp'] = pd.to_datetime(df_user_liquidations['timestamp'], unit='s', origin='unix').dt.date
df_user_liquidations = pd.merge(df_user_liquidations, df_token_usd, how='left', left_on=['timestamp','collateralReserve.symbol'], right_on=['timestamp','token']) \
    .rename(columns={"token_usd": "collateral_token_usd"}).drop(['token'], axis=1) \
        .merge(df_token_usd, how='left', left_on=['timestamp','principalReserve.symbol'], right_on=['timestamp','token']).rename(columns={"token_usd": "principal_token_usd"}).drop(['token'], axis=1)
# collateral
df_user_liquidations['collateralLiquidationAmountTokens'] = (df_user_liquidations['collateralAmount'].astype('float') / (10**df_user_liquidations['collateralReserve.decimals']))
# df_user_liquidations['collateralLiquidationAmountETH'] = (df_user_liquidations['collateralLiquidationAmountTokens'] * (df_user_liquidations['liquidation.collateralReserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_liquidations['collateralLiquidationAmountUSD'] = df_user_liquidations['collateralLiquidationAmountTokens'] * df_user_liquidations['collateral_token_usd']

# outstanding loan
df_user_liquidations['loanDefaultAmountTokens'] = (df_user_liquidations['principalAmount'].astype('float') / (10**df_user_liquidations['principalReserve.decimals']))
# df_user_liquidations['loanDefaultAmountETH'] = (df_user_liquidations['loanDefaultAmountTokens'] * (df_user_liquidations['liquidation.principalReserve.price.priceInEth'].astype('float') /  (10**18)))
df_user_liquidations['loanDefaultAmountUSD'] = df_user_liquidations['loanDefaultAmountTokens'] * df_user_liquidations['principal_token_usd']
# #summarize each
df_user_liquidate_collateral_sum = df_user_liquidations[['user.id','collateralLiquidationAmountUSD']].groupby(['user.id']).sum().reset_index()
df_user_liquidate_loan_sum = df_user_liquidations[['user.id','loanDefaultAmountUSD']].groupby(['user.id']).sum().reset_index()

In [672]:
# user assets
df_user_assets = pd.merge(df_users, df_user_deposit_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
        .merge(df_user_withdrawal_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                .merge(df_user_liquidate_collateral_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                        .merge(df_user_aToken_sum, how="left", left_on=["user.id"], right_on=["user.id"])
df_user_assets.fillna(0,inplace=True)
df_user_assets

,user.id,depositAmountUSD,withdrawalAmountUSD,collateralLiquidationAmountUSD,aTokenAmountUSD
0,0x2e771628e765c04a06252801177332d078a19a21,"2,484,631.42","773,385.96","1,351,425.45",0.00
1,0x991f7454dbbcfd3506ccb47069a988dc8d7b32df,591.81,"1,274.17",0.00,0.00
2,0xe6354ed5bc4b393a5aad09f21c46e101e692d447,"257,764,911.34","274,465,498.00",0.00,"1,380,589.63"
3,0x66329fdd4042928bfcab60b179e1538d56eeeeee,"7,254.76","7,803.37",0.00,0.00
4,0x1be5d71f2da660bfdee8012ddc58d024448a0a59,"30,312,623.29","28,591,145.71",0.00,"65,120.06"
...,...,...,...,...,...
245,0xdd21fec3bd902820d70d708edc6aa986b25f4380,"84,204.40","21,188.93",0.00,0.00
246,0x4086e3e1e99a563989a9390facff553a4f29b6ee,"76,075,875.45","82,465,688.67",0.00,0.00
247,0xac5660b801e73a6b7f29e75c0b1fd35f09227ab9,6.00,0.00,0.00,2.99
248,0x97319017b05e6eed7cfda06171979a2772dab2f1,1.75,0.00,0.00,1.33


In [673]:
# user liabilities
df_user_liabilities = pd.merge(df_users, df_user_borrow_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
        .merge(df_user_flash_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                .merge(df_user_repay_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                        .merge(df_user_liquidate_loan_sum, how="left", left_on=["user.id"], right_on=["user.id"])
df_user_liabilities.fillna(0,inplace=True)
# assume unpaid borrow is outstanding
df_user_liabilities['outstandingDebt'] = df_user_liabilities['borrowAmountUSD'] - df_user_liabilities['flashAmountUSD'] - df_user_liabilities['repayAmountUSD'] - df_user_liabilities['loanDefaultAmountUSD']
df_user_liabilities['outstandingDebt'] = df_user_liabilities['outstandingDebt'].apply(lambda x : x if x > 0 else 0)
df_user_liabilities

,user.id,borrowAmountUSD,flashAmountUSD,repayAmountUSD,loanDefaultAmountUSD,outstandingDebt
0,0x2e771628e765c04a06252801177332d078a19a21,"2,041,853.30","2,041,853.30","1,396,572.20","1,041,744.76",0
1,0x991f7454dbbcfd3506ccb47069a988dc8d7b32df,0.00,0.00,0.00,0.00,0
2,0xe6354ed5bc4b393a5aad09f21c46e101e692d447,0.00,0.00,0.00,0.00,0
3,0x66329fdd4042928bfcab60b179e1538d56eeeeee,0.00,0.00,0.00,0.00,0
4,0x1be5d71f2da660bfdee8012ddc58d024448a0a59,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...
245,0xdd21fec3bd902820d70d708edc6aa986b25f4380,"16,388.87","16,388.87","16,456.57",0.00,0
246,0x4086e3e1e99a563989a9390facff553a4f29b6ee,"53,389,427.42","53,389,427.42","53,061,889.03",0.00,0
247,0xac5660b801e73a6b7f29e75c0b1fd35f09227ab9,0.00,0.00,0.00,0.00,0
248,0x97319017b05e6eed7cfda06171979a2772dab2f1,0.00,0.00,0.00,0.00,0


In [704]:
# user assets and liabilities
df_user_asset_liabilities = pd.merge(df_users, df_user_deposit_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
        .merge(df_user_withdrawal_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                .merge(df_user_liquidate_collateral_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                        .merge(df_user_aToken_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                            .merge(df_user_borrow_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                                .merge(df_user_flash_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                                    .merge(df_user_repay_sum, how="left", left_on=["user.id"], right_on=["user.id"]) \
                                        .merge(df_user_liquidate_loan_sum, how="left", left_on=["user.id"], right_on=["user.id"])
# assume unpaid borrow is outstanding
df_user_asset_liabilities['outstandingDebt'] = df_user_asset_liabilities['borrowAmountUSD'] - df_user_liabilities['flashAmountUSD'] - df_user_asset_liabilities['repayAmountUSD'] - df_user_asset_liabilities['loanDefaultAmountUSD']
df_user_asset_liabilities['outstandingDebt'] = df_user_asset_liabilities['outstandingDebt'].apply(lambda x : x if x > 0 else 0)
df_user_asset_liabilities.fillna(0,inplace=True)
df_user_asset_liabilities

,user.id,depositAmountUSD,withdrawalAmountUSD,collateralLiquidationAmountUSD,aTokenAmountUSD,borrowAmountUSD,flashAmountUSD,repayAmountUSD,loanDefaultAmountUSD,outstandingDebt
0,0x2e771628e765c04a06252801177332d078a19a21,"2,484,631.42","773,385.96","1,351,425.45",0.00,"2,041,853.30","2,041,853.30","1,396,572.20","1,041,744.76",0
1,0x991f7454dbbcfd3506ccb47069a988dc8d7b32df,591.81,"1,274.17",0.00,0.00,0.00,0.00,0.00,0.00,0
2,0xe6354ed5bc4b393a5aad09f21c46e101e692d447,"257,764,911.34","274,465,498.00",0.00,"1,380,589.63",0.00,0.00,0.00,0.00,0
3,0x66329fdd4042928bfcab60b179e1538d56eeeeee,"7,254.76","7,803.37",0.00,0.00,0.00,0.00,0.00,0.00,0
4,0x1be5d71f2da660bfdee8012ddc58d024448a0a59,"30,312,623.29","28,591,145.71",0.00,"65,120.06",0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...
245,0xdd21fec3bd902820d70d708edc6aa986b25f4380,"84,204.40","21,188.93",0.00,0.00,"16,388.87","16,388.87","16,456.57",0.00,0
246,0x4086e3e1e99a563989a9390facff553a4f29b6ee,"76,075,875.45","82,465,688.67",0.00,0.00,"53,389,427.42","53,389,427.42","53,061,889.03",0.00,0
247,0xac5660b801e73a6b7f29e75c0b1fd35f09227ab9,6.00,0.00,0.00,2.99,0.00,0.00,0.00,0.00,0
248,0x97319017b05e6eed7cfda06171979a2772dab2f1,1.75,0.00,0.00,1.33,0.00,0.00,0.00,0.00,0


In [675]:
# get list of aave users with ENS resolved addresses
# first pull from ENS subgraph
ens_graph_url = "https://api.thegraph.com/subgraphs/name/ensdomains/ens"
ens_transport = AIOHTTPTransport(url=ens_graph_url)

# latest 100 transactions
# borrow asynchronous subgraph call - get data
async def ensResolversMain(userList, transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query userENSdomains {{
                domains(first: 1, where: {{resolvedAddress: "{userList}"}}) {{
                    id
                    createdAt
                    name
                    labelName
                    labelhash
                    subdomainCount
                    resolvedAddress {{
                        id
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get all address latest resolved domain
async def ensResolverCallMain(userList,transport):
    result_json = await ensResolversMain(userList,transport)
    return result_json["domains"]

In [676]:
# ens resolved addresses
# empty list to capture main graphql calls
ens_resolver_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await ensResolverCallMain(user, ens_transport)
    # iterate the params
    ens_resolver_list.append(this_list)

# flaten list
ens_resolver_list = [item for sublist in ens_resolver_list for item in sublist]
print(f'combined list is {len(ens_resolver_list)} records')

# convert list of json to df
df_ens_resolvers = pd.json_normalize(ens_resolver_list)

combined list is 63 records


In [705]:
# merge aave user financials with ens resolver list
# convert data types
df_ens_resolvers['createdAt'] = pd.to_datetime(df_ens_resolvers['createdAt'], unit='s')
df_user_asset_liabilities = pd.merge(df_user_asset_liabilities, df_ens_resolvers[['resolvedAddress.id','subdomainCount','createdAt']], how='left', left_on='user.id', right_on='resolvedAddress.id')

In [706]:
# make ENS category
df_user_asset_liabilities['ens_resolver'] = np.where(df_user_asset_liabilities['resolvedAddress.id'].isnull(), 0, 1)
df_user_asset_liabilities['ens_resolver'] = df_user_asset_liabilities['ens_resolver'].astype("category")

# change subdomainCount to int
df_user_asset_liabilities['ens_subdomain_count'] = pd.to_numeric(df_user_asset_liabilities['subdomainCount'].fillna(0), downcast='integer')

# number of days since resolver set
df_user_asset_liabilities['ens_resolver_age_days'] = (datetime.datetime.today().date() - df_user_asset_liabilities['createdAt'].dt.date).dt.days
df_user_asset_liabilities['ens_resolver_age_days'] = pd.to_numeric(df_user_asset_liabilities['ens_resolver_age_days'].fillna(0), downcast='integer')

df_user_asset_liabilities.drop(['resolvedAddress.id','createdAt','subdomainCount'], axis=1, inplace=True)

In [682]:
# lens profiles

# first pull from lens subgraph
lens_graph_url = "https://api.lens.dev/"
lens_transport = AIOHTTPTransport(url=lens_graph_url)

# latest 100 transactions
# borrow asynchronous subgraph call - get data
async def lensProfileMain(userList, transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query Profiles {{
                profiles(request: {{ ownedBy: [{userList}] }}) {{
                    items {{
                        handle
                        ownedBy
                        stats {{
                            totalFollowers
                            totalFollowing
                            totalPublications
                            totalCollects
                        }}
                    }}
                    pageInfo {{
                        prev
                        next
                        totalCount
                    }}
                }}
            }}
        '''.format(userList = userList, transport = transport)
        
        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get all address latest resolved domain
async def lensProfileCallMain(userList,transport):
    result_json = await lensProfileMain(userList,transport)
    return result_json["profiles"]

In [683]:
# lens profiles
lens_list = await lensProfileCallMain(latest_user_query_string, lens_transport)

df_lens_profiles = pd.json_normalize(lens_list, 'items')

In [707]:
# merge lens data into aave financial data
# check if 'DAO' is in handle
df_lens_profiles['lens_handle_contains_dao'] = df_lens_profiles['handle'].str.contains('dao').astype('int8')

# merge lens data into aave financial data
df_lens_profiles['ownedBy'] = df_lens_profiles['ownedBy'].str.lower()
df_user_asset_liabilities = pd.merge(df_user_asset_liabilities, df_lens_profiles, how='left', left_on='user.id', right_on='ownedBy')

In [708]:
# make lens ownedBy into category
df_user_asset_liabilities['lens_profile_owner'] = np.where(df_user_asset_liabilities['ownedBy'].isnull(), 0, 1)
df_user_asset_liabilities['lens_profile_owner'] = df_user_asset_liabilities['lens_profile_owner'].astype("category")
df_user_asset_liabilities.drop(['handle','ownedBy'], axis=1, inplace=True)

In [709]:
# rename lens columns for readability
df_user_asset_liabilities.rename(columns={"stats.totalFollowers": "lens.totalFollowers", "stats.totalFollowing":"lens.totalFollowing","stats.totalPublications": "lens.totalPublications", "stats.totalCollects": "lens.totalCollects"}, inplace=True)

In [687]:
# Safe multisig signer

# get list of aave users with ENS resolved addresses
# first pull from ENS subgraph
safe_graph_url = "https://api.thegraph.com/subgraphs/name/gjeanmart/gnosis-safe-polygon"
safe_transport = AIOHTTPTransport(url=safe_graph_url)

# latest 100 transactions
# borrow asynchronous subgraph call - get data
async def safeSignerMain(userList, transport):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        # build the graphQL script
        query_script = '''
            query safeWallets {{
                wallets(where: {{threshold_gte: "2", owners_contains: ["{userList}"]}}, first: 1) {{
                    id
                    owners
                    creator
                    threshold
                }}
            }}
        '''.format(userList = userList, transport = transport)

        query = gql(query_script)

        result = await session.execute(query)
        return result

# call main - get all address latest resolved domain
async def safeSignerCallMain(userList,transport):
    result_json = await safeSignerMain(userList,transport)
    return result_json["wallets"]

In [698]:
# safe signer
# empty list to capture main graphql calls
safe_signer_list = []

# get k rows by looping n times
for user in latest_users:
    this_list = await safeSignerCallMain(user, safe_transport)
    # iterate the params
    safe_signer_list.append(this_list)

# flaten list
safe_signer_list = [item for sublist in safe_signer_list for item in sublist]
print(f'combined list is {len(safe_signer_list)} records')

# convert list of json to df
df_safe_signers = pd.json_normalize(safe_signer_list, 'owners', 'threshold')

# rename columns and fix var type
df_safe_signers.columns = ['safe_owner','safe_multisig_count']
df_safe_signers['safe_multisig_count'] = pd.to_numeric(df_safe_signers['safe_multisig_count'], downcast='integer')

combined list is 5 records


In [710]:
# merge with aave user financials
df_user_asset_liabilities = pd.merge(df_user_asset_liabilities, df_safe_signers, how='left', left_on='user.id', right_on="safe_owner")

In [711]:
# make safe owner category
df_user_asset_liabilities['safe_owner'] = np.where(df_user_asset_liabilities['safe_owner'].isnull(), 0, 1)
df_user_asset_liabilities['safe_owner'] = df_user_asset_liabilities['safe_owner'].astype("category")

In [715]:
# ratio of liquidated collateral to total borrow
df_user_asset_liabilities['liquidated_principal_to_borrows'] = df_user_asset_liabilities['collateralLiquidationAmountUSD'] / df_user_asset_liabilities['borrowAmountUSD']
df_user_asset_liabilities.fillna(0)

,user.id,depositAmountUSD,withdrawalAmountUSD,collateralLiquidationAmountUSD,aTokenAmountUSD,borrowAmountUSD,flashAmountUSD,repayAmountUSD,loanDefaultAmountUSD,outstandingDebt,...,ens_resolver_age_days,lens.totalFollowers,lens.totalFollowing,lens.totalPublications,lens.totalCollects,lens_handle_contains_dao,lens_profile_owner,safe_owner,safe_multisig_count,liquidated_principal_to_borrows
0,0x2e771628e765c04a06252801177332d078a19a21,"2,484,631.42","773,385.96","1,351,425.45",0.00,"2,041,853.30","2,041,853.30","1,396,572.20","1,041,744.76",0,...,391,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.66
1,0x991f7454dbbcfd3506ccb47069a988dc8d7b32df,591.81,"1,274.17",0.00,0.00,0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
2,0xe6354ed5bc4b393a5aad09f21c46e101e692d447,"257,764,911.34","274,465,498.00",0.00,"1,380,589.63",0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
3,0x66329fdd4042928bfcab60b179e1538d56eeeeee,"7,254.76","7,803.37",0.00,0.00,0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
4,0x1be5d71f2da660bfdee8012ddc58d024448a0a59,"30,312,623.29","28,591,145.71",0.00,"65,120.06",0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0xdd21fec3bd902820d70d708edc6aa986b25f4380,"84,204.40","21,188.93",0.00,0.00,"16,388.87","16,388.87","16,456.57",0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
247,0x4086e3e1e99a563989a9390facff553a4f29b6ee,"76,075,875.45","82,465,688.67",0.00,0.00,"53,389,427.42","53,389,427.42","53,061,889.03",0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
248,0xac5660b801e73a6b7f29e75c0b1fd35f09227ab9,6.00,0.00,0.00,2.99,0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00
249,0x97319017b05e6eed7cfda06171979a2772dab2f1,1.75,0.00,0.00,1.33,0.00,0.00,0.00,0.00,0,...,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00


In [716]:
# ratio of total borrowing (USD) to total repayments (USD)
df_user_asset_liabilities['borrowings_to_repayments'] = df_user_asset_liabilities['borrowAmountUSD'] / df_user_asset_liabilities['repayAmountUSD']

In [717]:
# prior repayments made
df_user_asset_liabilities['repay_history'] = np.where(df_user_asset_liabilities['repayAmountUSD'] > 0, 1, 0)
df_user_asset_liabilities['repay_history'] = df_user_asset_liabilities['repay_history'].astype('category')

In [718]:
# get target variable 
# if user has had liquidated principal then loan adherence is 0. 
df_user_asset_liabilities['result'] = np.where(df_user_asset_liabilities['loanDefaultAmountUSD'] > 0, 0, 1)

In [ ]:
# data exploration before cutting final analysis dataset

In [719]:
# top ten depositors
df_user_asset_liabilities.sort_values(by='depositAmountUSD', ascending=False).head(10)

,user.id,depositAmountUSD,withdrawalAmountUSD,collateralLiquidationAmountUSD,aTokenAmountUSD,borrowAmountUSD,flashAmountUSD,repayAmountUSD,loanDefaultAmountUSD,outstandingDebt,...,lens.totalPublications,lens.totalCollects,lens_handle_contains_dao,lens_profile_owner,safe_owner,safe_multisig_count,liquidated_principal_to_borrows,borrowings_to_repayments,repay_history,result
8,0xbb7d75be4dc8eb15ff90422137c0a5bcbd316953,"282,852,085.62","283,647,010.42",0.00,"44,320.07",0.00,0.00,0.00,0.00,0,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,1
2,0xe6354ed5bc4b393a5aad09f21c46e101e692d447,"257,764,911.34","274,465,498.00",0.00,"1,380,589.63",0.00,0.00,0.00,0.00,0,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,1
247,0x4086e3e1e99a563989a9390facff553a4f29b6ee,"76,075,875.45","82,465,688.67",0.00,0.00,"53,389,427.42","53,389,427.42","53,061,889.03",0.00,0,...,NaN,NaN,NaN,0,0,NaN,0.00,1.01,1,1
4,0x1be5d71f2da660bfdee8012ddc58d024448a0a59,"30,312,623.29","28,591,145.71",0.00,"65,120.06",0.00,0.00,0.00,0.00,0,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,1
198,0x2b67a3c0b90f6ae4394210692f69968d02970126,"28,014,598.05","33,941,546.84",0.00,0.00,"13,957,975.30","13,957,975.30","14,242,767.65",0.00,0,...,NaN,NaN,NaN,0,0,NaN,0.00,0.98,1,1
229,0x7e6f6621388047c8a481d963210b514dbd5ea1b9,"26,242,607.75","26,242,688.44",0.00,0.00,0.00,0.00,0.00,0.00,0,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,1
175,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,"15,994,912.59","15,995,843.24",0.00,0.00,"7,532,034.97","7,532,034.97","7,554,174.26",0.00,0,...,NaN,NaN,NaN,0,0,NaN,0.00,1.00,1,1
176,0x3eb6fc6c786a1284f07dc68cba4f7e95f4ca50d1,"12,496,784.10","12,496,788.24",0.00,0.00,"4,906,707.86","4,906,707.86","4,907,167.74",0.00,0,...,NaN,NaN,NaN,0,0,NaN,0.00,1.00,1,1
33,0x738726961dabef11514ec9b75e0665ee0050ebc4,"6,835,395.02","4,382,832.35","4,349,763.02",0.00,"7,579,062.94","7,579,062.94","4,999,523.10","3,474,597.10",0,...,NaN,NaN,NaN,0,0,NaN,0.57,1.52,1,0
29,0x9f3c2254414c852b83c727b257b6eab9418cf914,"3,919,254.44","3,709,701.43","299,672.86",0.00,"2,195,748.60","2,195,748.60","2,001,154.28","234,214.80",0,...,NaN,NaN,NaN,0,0,NaN,0.14,1.10,1,0


In [720]:
# count of users with deposits
df_user_asset_liabilities['depositAmountUSD'][df_user_asset_liabilities['depositAmountUSD']>0].count()

235

In [721]:
# count of users with borrowings
df_user_asset_liabilities['borrowAmountUSD'][df_user_asset_liabilities['borrowAmountUSD']>0].count()

80

In [722]:
# count flashloans
df_user_asset_liabilities['flashAmountUSD'][df_user_asset_liabilities['flashAmountUSD']>0].count()

80

In [723]:
# count of users with liquidated principal
df_user_asset_liabilities['loanDefaultAmountUSD'][df_user_asset_liabilities['loanDefaultAmountUSD']>0].count()

32

In [724]:
# liquidated borrowers
df_user_asset_liabilities.sort_values(by='loanDefaultAmountUSD', ascending=False).head(35)

,user.id,depositAmountUSD,withdrawalAmountUSD,collateralLiquidationAmountUSD,aTokenAmountUSD,borrowAmountUSD,flashAmountUSD,repayAmountUSD,loanDefaultAmountUSD,outstandingDebt,...,lens.totalPublications,lens.totalCollects,lens_handle_contains_dao,lens_profile_owner,safe_owner,safe_multisig_count,liquidated_principal_to_borrows,borrowings_to_repayments,repay_history,result
173,0x165be24e4678d7bfa9fc8d58a275dd93b98aa008,"28,257.47","7,537.94","35,310,579.88","2,582.66","10,362.71","10,362.71","99,287.98","35,210,284.13",0,...,NaN,NaN,NaN,0,0,NaN,"3,407.46",0.10,1,0
33,0x738726961dabef11514ec9b75e0665ee0050ebc4,"6,835,395.02","4,382,832.35","4,349,763.02",0.00,"7,579,062.94","7,579,062.94","4,999,523.10","3,474,597.10",0,...,NaN,NaN,NaN,0,0,NaN,0.57,1.52,1,0
0,0x2e771628e765c04a06252801177332d078a19a21,"2,484,631.42","773,385.96","1,351,425.45",0.00,"2,041,853.30","2,041,853.30","1,396,572.20","1,041,744.76",0,...,NaN,NaN,NaN,0,0,NaN,0.66,1.46,1,0
205,0x19654fbe6b0d05a68c3fadb833ce1fdc66f98b77,"858,008.57","154,645.64","1,176,369.21",0.00,"814,003.02","814,003.02","23,655.45","1,019,874.79",0,...,NaN,NaN,NaN,0,0,NaN,1.45,34.41,1,0
41,0xe152bea6dddfe873c44ea076ee079fe2df266ced,"231,224.93","29,783.47","726,316.92",603.89,"654,814.09","654,814.09","115,296.84","607,716.11",0,...,NaN,NaN,NaN,0,0,NaN,1.11,5.68,1,0
46,0x440efbba3f3d971fd8411c580197ecb53ef71f87,"2,030,697.51","1,497,008.34","581,696.98",0.00,"1,333,284.59","1,333,284.59","1,030,013.11","415,432.90",0,...,NaN,NaN,NaN,0,0,NaN,0.44,1.29,1,0
89,0xae2c4abffbae5afde60467d26d50e98930e1e2dd,"621,483.26","375,355.64","462,981.48",0.00,"446,282.72","446,282.72","145,188.69","323,369.50",0,...,NaN,NaN,NaN,0,0,NaN,1.04,3.07,1,0
29,0x9f3c2254414c852b83c727b257b6eab9418cf914,"3,919,254.44","3,709,701.43","299,672.86",0.00,"2,195,748.60","2,195,748.60","2,001,154.28","234,214.80",0,...,NaN,NaN,NaN,0,0,NaN,0.14,1.10,1,0
167,0x69dcb0a3ab51c7adaf110e6f119d886989b53ec8,"1,475,944.38","1,065,387.62","150,533.43",0.00,"269,271.15","269,271.15","19,554.17","117,396.35",0,...,NaN,NaN,NaN,0,0,NaN,0.56,13.77,1,0
160,0x8ed591ff208e9116d558b15c34bfd9c2562b41e4,"35,447.59","49,595.52","120,927.07",0.00,"94,335.63","94,335.63",0.00,"108,487.17",0,...,NaN,NaN,NaN,0,0,NaN,1.28,inf,0,0


In [741]:
# create analysis dataset
df_final = df_user_asset_liabilities
df_final = df_final.fillna(0)

In [742]:
# drop unused features
df_final.drop(['withdrawalAmountUSD','collateralLiquidationAmountUSD','aTokenAmountUSD','loanDefaultAmountUSD','outstandingDebt','liquidated_principal_to_borrows','repayAmountUSD'], axis=1, inplace=True)

In [743]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   user.id                   251 non-null    object  
 1   depositAmountUSD          251 non-null    float64 
 2   borrowAmountUSD           251 non-null    float64 
 3   flashAmountUSD            251 non-null    float64 
 4   ens_resolver              251 non-null    category
 5   ens_subdomain_count       251 non-null    int8    
 6   ens_resolver_age_days     251 non-null    int16   
 7   lens.totalFollowers       251 non-null    float64 
 8   lens.totalFollowing       251 non-null    float64 
 9   lens.totalPublications    251 non-null    float64 
 10  lens.totalCollects        251 non-null    float64 
 11  lens_handle_contains_dao  251 non-null    float64 
 12  lens_profile_owner        251 non-null    category
 13  safe_owner                251 non-null    category

In [744]:
# slit features and target
x = df_final[['depositAmountUSD','borrowAmountUSD','flashAmountUSD','ens_resolver','ens_subdomain_count','ens_resolver_age_days','lens.totalFollowers','lens.totalFollowing','lens.totalPublications','lens.totalCollects','lens_handle_contains_dao','lens_profile_owner','safe_owner','safe_multisig_count','repay_history']]
y = df_final['result']

In [745]:
# Split dataset into training set and test set
# 80% training and 20% test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1) 

In [746]:
# create decision tree classifer object
clf = DecisionTreeClassifier()

# train decision tree classifer
clf = clf.fit(x_train,y_train)

# predict the response for test dataset
y_pred = clf.predict(x_test)

In [747]:
# model accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7843137254901961


In [748]:
# get importance
importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.16319
Feature: 1, Score: 0.55409
Feature: 2, Score: 0.14576
Feature: 3, Score: 0.00000
Feature: 4, Score: 0.00000
Feature: 5, Score: 0.04209
Feature: 6, Score: 0.00000
Feature: 7, Score: 0.00000
Feature: 8, Score: 0.02624
Feature: 9, Score: 0.00000
Feature: 10, Score: 0.00000
Feature: 11, Score: 0.00000
Feature: 12, Score: 0.00000
Feature: 13, Score: 0.00000
Feature: 14, Score: 0.06863


In [749]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   depositAmountUSD          251 non-null    float64 
 1   borrowAmountUSD           251 non-null    float64 
 2   flashAmountUSD            251 non-null    float64 
 3   ens_resolver              251 non-null    category
 4   ens_subdomain_count       251 non-null    int8    
 5   ens_resolver_age_days     251 non-null    int16   
 6   lens.totalFollowers       251 non-null    float64 
 7   lens.totalFollowing       251 non-null    float64 
 8   lens.totalPublications    251 non-null    float64 
 9   lens.totalCollects        251 non-null    float64 
 10  lens_handle_contains_dao  251 non-null    float64 
 11  lens_profile_owner        251 non-null    category
 12  safe_owner                251 non-null    category
 13  safe_multisig_count       251 non-null    float64 

In [750]:
df_final['depositAmountUSD'].corr(df_final['result'])

0.037483274923552724

In [751]:
df_final['flashAmountUSD'].corr(df_final['result'])

-0.012136625765001071